In [33]:
class FashionMNISTLoader(BaseDataset):
    def __init__(self,
                 root: str='/net/leksai/data/FashionMNIST',
                 label_normal: tuple=(0,),
                 label_abnormal: tuple=(),  # If unsupervised, do not specify
                 ratio_abnormal: float=0.1):
        super().__init__(root)

        # Initialization
        self.root = root
        self.label_normal = label_normal
        self.label_abnormal = label_abnormal
        self.ratio_abnormal = ratio_abnormal

        # Read in initial Full Set
        # Add in download=True if you haven't downloaded yet
        print('Loading dataset for you!')
        train_set = FashionMNISTDataset(root=root, train=True, transform=transforms.ToTensor(), download=True)
        test_set = FashionMNISTDataset(root=root, train=False, transform=transforms.ToTensor(), download=True)
        print('Almost loaded!')

        # Get the labels for classes intended to use
        y_train = train_set.targets.cpu().data.numpy()
        y_test = test_set.targets.cpu().data.numpy()

        # Get the indices for classes intended to use
        train_idx = self.get_idx(y_train, label_normal, label_abnormal, ratio_abnormal, True)
        test_idx = self.get_idx(y_test, label_normal, label_abnormal, ratio_abnormal, False)

        # Get the subset
        self.train_set = Subset(train_set, train_idx)
        self.test_set = Subset(test_set, test_idx)

    def get_idx(self, y, label_normal, label_abnormal, ratio_abnormal, train):
        """
        Creat a numpy list of indices of label_ in labels.
        Inputs:
            y (np.array): dataset.targets.cpu().data.numpy()
            label_normal (tuple): e.g. (0,)
            label_abnormal (tuple): e.g. (1,)
            ratio_abnormal (float): e.g. 0.1
            train (bool): True / False
        """
        idx_normal = np.argwhere(np.isin(y, label_normal)).flatten()

        if label_abnormal:
            idx_abnormal = np.argwhere(np.isin(y, label_abnormal)).flatten()
            np.random.shuffle(idx_abnormal)
            if train:
                idx_abnormal = idx_abnormal[:int(len(idx_abnormal) * ratio_abnormal)]
            idx_all = np.hstack((idx_normal, idx_abnormal))
        else:
            idx_all = idx_normal
        return idx_all

    def loaders(self,
                batch_size: int,
                shuffle_train=True,
                shuffle_test=False,
                num_workers: int = 0) -> (DataLoader, DataLoader):
        train_loader = DataLoader(dataset=self.train_set,
                                  batch_size=batch_size,
                                  shuffle=shuffle_train,
                                  num_workers=num_workers,
                                  drop_last=True)
        test_loader = DataLoader(dataset=self.test_set,
                                 batch_size=batch_size,
                                 shuffle=shuffle_test,
                                 num_workers=num_workers,
                                 drop_last=False)
        return train_loader, test_loader



In [37]:
dataset = FashionMNISTLoader()

Loading dataset for you!
Almost loaded!


In [39]:
_, loader = dataset.loaders(batch_size=batch_size_loading,
                         num_workers=num_workers)
for data in loader:
    X, y, idx = data
    break

In [40]:
X

tensor([[[[0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
           0.0000, 0.2039, 0.7529, 0.4510, 0.1059, 0.0039, 0.0039, 0.1216,
           0.5608, 0.7020, 0.1255, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
           0.0000, 0.0000, 0.0000, 0.0000],
          [0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0196, 0.4431,
           0.7255, 0.8471, 0.7451, 0.8980, 0.9294, 0.8627, 0.8510, 0.9490,
           0.7882, 0.7490, 0.8706, 0.6549, 0.3412, 0.0000, 0.0000, 0.0000,
           0.0000, 0.0000, 0.0000, 0.0000],
          [0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.1961, 0.6863, 0.7608,
           0.7412, 0.7294, 0.7255, 0.7333, 0.7804, 0.7373, 0.7216, 0.7569,
           0.7255, 0.7373, 0.7176, 0.7333, 0.7569, 0.8235, 0.0588, 0.0000,
           0.0000, 0.0000, 0.0000, 0.0000],
          [0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.7804, 0.7608, 0.7098,
           0.7373, 0.7451, 0.7412, 0.7373, 0.7529, 0.7804, 0.7725, 0.7647,
           0.7529, 0.7490, 0.7373, 0.7216, 

In [28]:
from PIL import Image
from abc import ABC, abstractmethod
from torch.utils.data import Subset
from torch.utils.data import DataLoader
from torch.utils.data import ConcatDataset
from torchvision.datasets import KMNIST
from torchvision.datasets import FashionMNIST

import torch
import torchvision
import numpy as np
import torchvision.transforms as transforms


# #########################################################################
# 1. Base Dataset
# #########################################################################
class BaseDataset(ABC):
    def __init__(self, root: str):
        super().__init__()

        self.root = root
        self.label_normal = ()
        self.label_abnormal = ()
        self.train_set = None
        self.test_set = None

    @abstractmethod
    def loaders(self,
                batch_size: int,
                shuffle_train=True,
                shuffle_test=False,
                num_workers: int = 0):
        pass

    def __repr__(self):
        return self.__class__.__name__

In [41]:
class KMNISTDataset(KMNIST):
    """
    Add an index to get item.
    """
    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)

    def __getitem__(self, index):
        img, target = self.data[index], int(self.targets[index])
        img = Image.fromarray(img.numpy(), mode='L')
        transform = transforms.ToTensor()
        img = transform(img)
        return img, int(target), index

In [42]:
root = '/net/leksai/data/KMNIST'

In [61]:
root

'/net/leksai/data/KMNIST'

In [62]:
train_set = KMNISTDataset(root=root, train=True, transform=transforms.ToTensor(), download=True)

In [63]:
test_set = KMNISTDataset(root=root, train=False, transform=transforms.ToTensor(), download=True)

In [73]:
train_loader = DataLoader(dataset=train_set,
                                  batch_size=128,
                                  shuffle=True,
                                  num_workers=num_workers,
                                  drop_last=True)

In [75]:
for data in train_loader:
    X, y, idx = data
    break

In [69]:
untransform = transforms.ToPILImage()
X_mean = torch.mean(X, axis=0)
img = untransform(X_mean);  display(img)

In [70]:
X.shape

torch.Size([1, 1, 28, 28])

In [77]:
X_0 = X[2]
img = untransform(X_0);  display(img)

In [105]:
X[0].shape

torch.Size([1, 28, 28])

In [78]:
class KMNISTLoader(BaseDataset):
    def __init__(self,
                 root: str='/net/leksai/data/KMNIST',
                 label_normal: tuple=(0,),
                 label_abnormal: tuple=(),  # If unsupervised, do not specify
                 ratio_abnormal: float=0.1):
        super().__init__(root)

        # Initialization
        self.root = root
        self.label_normal = label_normal
        self.label_abnormal = label_abnormal
        self.ratio_abnormal = ratio_abnormal

        # Read in initial Full Set
        # Add in download=True if you haven't downloaded yet
        print('Loading dataset for you!')
        train_set = KMNISTDataset(root=root, train=True, transform=transforms.ToTensor(), download=True)
        test_set = KMNISTDataset(root=root, train=False, transform=transforms.ToTensor(), download=True)
        print('Almost loaded!')

        # Get the labels for classes intended to use
        y_train = train_set.targets.cpu().data.numpy()
        y_test = test_set.targets.cpu().data.numpy()

        # Get the indices for classes intended to use
        train_idx = self.get_idx(y_train, label_normal, label_abnormal, ratio_abnormal, True)
        test_idx = self.get_idx(y_test, label_normal, label_abnormal, ratio_abnormal, False)

        # Get the subset
        self.train_set = Subset(train_set, train_idx)
        self.test_set = Subset(test_set, test_idx)

    def get_idx(self, y, label_normal, label_abnormal, ratio_abnormal, train):
        """
        Creat a numpy list of indices of label_ in labels.
        Inputs:
            y (np.array): dataset.targets.cpu().data.numpy()
            label_normal (tuple): e.g. (0,)
            label_abnormal (tuple): e.g. (1,)
            ratio_abnormal (float): e.g. 0.1
            train (bool): True / False
        """
        idx_normal = np.argwhere(np.isin(y, label_normal)).flatten()

        if label_abnormal:
            idx_abnormal = np.argwhere(np.isin(y, label_abnormal)).flatten()
            np.random.shuffle(idx_abnormal)
            if train:
                idx_abnormal = idx_abnormal[:int(len(idx_abnormal) * ratio_abnormal)]
            idx_all = np.hstack((idx_normal, idx_abnormal))
        else:
            idx_all = idx_normal
        return idx_all

    def loaders(self,
                batch_size: int,
                shuffle_train=True,
                shuffle_test=False,
                num_workers: int = 0) -> (DataLoader, DataLoader):
        train_loader = DataLoader(dataset=self.train_set,
                                  batch_size=batch_size,
                                  shuffle=shuffle_train,
                                  num_workers=num_workers,
                                  drop_last=True)
        test_loader = DataLoader(dataset=self.test_set,
                                 batch_size=batch_size,
                                 shuffle=shuffle_test,
                                 num_workers=num_workers,
                                 drop_last=False)
        return train_loader, test_loader



In [79]:
class KMNISTLoaderEval(BaseDataset):
    def __init__(self,
                 root: str='/net/leksai/data/KMNIST',
                 label: tuple=(),
                 test_eval: bool=False):
        super().__init__(root)

        # Initialization
        self.root = root
        self.label = label

        # Read in initial Full Set
        # Add in download=True if you haven't downloaded yet
        train_set = KMNISTDataset(root=root, train=True, transform=transforms.ToTensor(), download=True)
        test_set = KMNISTDataset(root=root, train=False, transform=transforms.ToTensor(), download=True)

        # Get the labels for classes intended to use
        y_train = train_set.targets.cpu().data.numpy()
        y_test = test_set.targets.cpu().data.numpy()

        # Get the indices for classes intended to use
        train_idx = self.get_idx(y_train, label)
        test_idx = self.get_idx(y_test, label)

        # Get the subset
        train_set = Subset(train_set, train_idx)
        test_set = Subset(test_set, test_idx)
        if test_eval:
            self.all_set = test_set
        else:
            self.all_set = ConcatDataset((train_set, test_set))

    def get_idx(self, y, label):
        """
        Creat a numpy list of indices of label_ in labels.
        Inputs:
            y (np.array): dataset.targets.cpu().data.numpy()
            label (tuple): e.g. (0,)
        """
        return np.argwhere(np.isin(y, label)).flatten()

    def loaders(self,
                batch_size: int,
                shuffle=False,
                num_workers: int = 0):
        all_loader = DataLoader(dataset=self.all_set,
                                batch_size=batch_size,
                                shuffle=shuffle,
                                num_workers=num_workers,
                                drop_last=False)
        return all_loader

In [100]:
dataset = KMNISTLoaderEval(label = (6,))

In [101]:
batch_size_loading = 7000

In [102]:
loader = dataset.loaders(batch_size=batch_size_loading,
                         num_workers=num_workers)
for data in loader:
    X, y, idx = data
    break
    
X_mean = torch.mean(X, axis=0)
img = untransform(X_mean); display(img)

In [93]:
X.shape

torch.Size([7000, 1, 28, 28])

In [35]:
"""
Title: fmnist_loader.py
Description: The loader classes for the FashionMNIST datasets
Author: Lek'Sai Ye, University of Chicago
"""

from PIL import Image
from abc import ABC, abstractmethod
from torch.utils.data import Subset
from torch.utils.data import DataLoader
from torch.utils.data import ConcatDataset
from torchvision.datasets import KMNIST

import torch
import torchvision
import numpy as np
import torchvision.transforms as transforms


# #########################################################################
# 1. Base Dataset
# #########################################################################
class BaseDataset(ABC):
    def __init__(self, root: str):
        super().__init__()

        self.root = root
        self.label_normal = ()
        self.label_abnormal = ()
        self.train_set = None
        self.test_set = None

    @abstractmethod
    def loaders(self,
                batch_size: int,
                shuffle_train=True,
                shuffle_test=False,
                num_workers: int = 0):
        pass

    def __repr__(self):
        return self.__class__.__name__


# #########################################################################
# 2. FashionMNIST Dataset
# #########################################################################
class FashionMNISTDataset(FashionMNIST):
    """
    Add an index to get item.
    """
    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)

    def __getitem__(self, index):
        img, target = self.data[index], int(self.targets[index])
        img = Image.fromarray(img.numpy(), mode='L')
        transform = transforms.ToTensor()
        img = transform(img)
        return img, int(target), index


# #########################################################################
# 2. FashionMNIST Loader for Training
# #########################################################################
class FashionMNISTLoader(BaseDataset):
    def __init__(self,
                 root: str='/net/leksai/data/FashionMNIST',
                 label_normal: tuple=(0,),
                 label_abnormal: tuple=(),  # If unsupervised, do not specify
                 ratio_abnormal: float=0.1):
        super().__init__(root)

        # Initialization
        self.root = root
        self.label_normal = label_normal
        self.label_abnormal = label_abnormal
        self.ratio_abnormal = ratio_abnormal

        # Read in initial Full Set
        # Add in download=True if you haven't downloaded yet
        print('Loading dataset for you!')
        train_set = FashionMNISTDataset(root=root, train=True, transform=transforms.ToTensor(), download=True)
        test_set = FashionMNISTDataset(root=root, train=False, transform=transforms.ToTensor(), download=True)
        print('Almost loaded!')

        # Get the labels for classes intended to use
        y_train = train_set.targets.cpu().data.numpy()
        y_test = test_set.targets.cpu().data.numpy()

        # Get the indices for classes intended to use
        train_idx = self.get_idx(y_train, label_normal, label_abnormal, ratio_abnormal, True)
        test_idx = self.get_idx(y_test, label_normal, label_abnormal, ratio_abnormal, False)

        # Get the subset
        self.train_set = Subset(train_set, train_idx)
        self.test_set = Subset(test_set, test_idx)

    def get_idx(self, y, label_normal, label_abnormal, ratio_abnormal, train):
        """
        Creat a numpy list of indices of label_ in labels.
        Inputs:
            y (np.array): dataset.targets.cpu().data.numpy()
            label_normal (tuple): e.g. (0,)
            label_abnormal (tuple): e.g. (1,)
            ratio_abnormal (float): e.g. 0.1
            train (bool): True / False
        """
        idx_normal = np.argwhere(np.isin(y, label_normal)).flatten()

        if label_abnormal:
            idx_abnormal = np.argwhere(np.isin(y, label_abnormal)).flatten()
            np.random.shuffle(idx_abnormal)
            if train:
                idx_abnormal = idx_abnormal[:int(len(idx_abnormal) * ratio_abnormal)]
            idx_all = np.hstack((idx_normal, idx_abnormal))
        else:
            idx_all = idx_normal
        return idx_all

    def loaders(self,
                batch_size: int,
                shuffle_train=True,
                shuffle_test=False,
                num_workers: int = 0) -> (DataLoader, DataLoader):
        train_loader = DataLoader(dataset=self.train_set,
                                  batch_size=batch_size,
                                  shuffle=shuffle_train,
                                  num_workers=num_workers,
                                  drop_last=True)
        test_loader = DataLoader(dataset=self.test_set,
                                 batch_size=batch_size,
                                 shuffle=shuffle_test,
                                 num_workers=num_workers,
                                 drop_last=False)
        return train_loader, test_loader


# #########################################################################
# 2. FashionMNIST Loader for Evaluation
# #########################################################################
class FashionMNISTLoaderEval(BaseDataset):
    def __init__(self,
                 root: str='/net/leksai/data/FashionMNIST',
                 label: tuple=(),
                 test_eval: bool=False):
        super().__init__(root)

        # Initialization
        self.root = root
        self.label = label

        # Read in initial Full Set
        # Add in download=True if you haven't downloaded yet
        train_set = FashionMNISTDataset(root=root, train=True, transform=transforms.ToTensor(), download=True)
        test_set = FashionMNISTDataset(root=root, train=False, transform=transforms.ToTensor(), download=True)

        # Get the labels for classes intended to use
        y_train = train_set.targets.cpu().data.numpy()
        y_test = test_set.targets.cpu().data.numpy()

        # Get the indices for classes intended to use
        train_idx = self.get_idx(y_train, label)
        test_idx = self.get_idx(y_test, label)

        # Get the subset
        train_set = Subset(train_set, train_idx)
        test_set = Subset(test_set, test_idx)
        if test_eval:
            self.all_set = test_set
        else:
            self.all_set = ConcatDataset((train_set, test_set))

    def get_idx(self, y, label):
        """
        Creat a numpy list of indices of label_ in labels.
        Inputs:
            y (np.array): dataset.targets.cpu().data.numpy()
            label (tuple): e.g. (0,)
        """
        return np.argwhere(np.isin(y, label)).flatten()

    def loaders(self,
                batch_size: int,
                shuffle=False,
                num_workers: int = 0):
        all_loader = DataLoader(dataset=self.all_set,
                                batch_size=batch_size,
                                shuffle=shuffle,
                                num_workers=num_workers,
                                drop_last=False)
        return all_loader
